In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import json
from datasets import Dataset, DatasetDict

# Dosya yollarını tanımlayın
train_path = "./train.json"
validation_path = "./validation.json"
test_path = "./test.json"

# JSON dosyalarını yükle
with open(train_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open(validation_path, "r", encoding="utf-8") as f:
    validation_data = json.load(f)

with open(test_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Dataset formatına dönüştür
train_dataset = Dataset.from_dict(train_data)
validation_dataset = Dataset.from_dict(validation_data)
test_dataset = Dataset.from_dict(test_data)

# Tüm veri setlerini birleştirmek için DatasetDict kullanabilirsiniz
dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

# Dataset kontrolü
print(dataset)



DatasetDict({
    train: Dataset({
        features: ['data'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['data'],
        num_rows: 1
    })
    test: Dataset({
        features: ['data'],
        num_rows: 1
    })
})


In [3]:
# İlk birkaç veriyi görüntüle
print(dataset["train"][0])  # İlk train örneği
print(dataset["validation"][0])  # İlk validation örneği
print(dataset["test"][0])  # İlk test örneği


{'data': {'paragraphs': [{'context': "Apse (apse; abscess; abscessus), irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur. Genellikle piyojen bakterilerin neden olduğu fokal bir yangı türüdür. Vücudun herhangi bir yerinde bir doku ya da organda oluşan apse, kendini ağrı, kızarıklık ve şişikler yaparak belli eder. Bazı apseler kendiliğinden dışarı açılır ve akar. Apse'nin açılmadığı durumlarda bu işi ameliyatla yapmak gerekir. Absenin mutlak tedavisi drenaj, yani boşaltılmasıdır. Cerrahi girişimden önce antibiyotik uygulanması sepsisi önler.", 'qas': [{'answers': [{'answer_start': 33, 'text': 'irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur'}], 'id': 1, 'question': 'Apse nedir ve nasıl oluşur?'}, {'answers': [{'answer_start': 119, 'text': 'piyojen bakterilerin'}], 'id': 2, 'question': 'Apse genellikle neyin neden olduğu bir yangı türüdür?'}, {'answers': [{'answer_start': 251, 'text': 'ağrı, kızarıklık ve şişikler'}], 'id': 3, 'question': 

In [9]:
def flatten_squad_format(data):
    flattened_data = {"context": [], "question": [], "answers": []}

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa["answers"]

                # Düzleştirilmiş veri setine ekleme
                flattened_data["context"].append(context)
                flattened_data["question"].append(question)
                flattened_data["answers"].append(answers)

    return flattened_data



In [10]:
# Veri setini işleme
flattened_train_data = flatten_squad_format(train_data)
flattened_validation_data = flatten_squad_format(validation_data)
flattened_test_data = flatten_squad_format(test_data)


In [19]:
print(flattened_train_data["context"][0])
print(flattened_train_data["question"][0])
print(flattened_train_data["answers"][0])

Apse (apse; abscess; abscessus), irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur. Genellikle piyojen bakterilerin neden olduğu fokal bir yangı türüdür. Vücudun herhangi bir yerinde bir doku ya da organda oluşan apse, kendini ağrı, kızarıklık ve şişikler yaparak belli eder. Bazı apseler kendiliğinden dışarı açılır ve akar. Apse'nin açılmadığı durumlarda bu işi ameliyatla yapmak gerekir. Absenin mutlak tedavisi drenaj, yani boşaltılmasıdır. Cerrahi girişimden önce antibiyotik uygulanması sepsisi önler.
Apse nedir ve nasıl oluşur?
[{'text': 'irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur', 'answer_start': 33}]


In [11]:
from datasets import Dataset, DatasetDict

# Dataset oluşturma
train_dataset = Dataset.from_dict(flattened_train_data)
validation_dataset = Dataset.from_dict(flattened_validation_data)
test_dataset = Dataset.from_dict(flattened_test_data)

# DatasetDict oluşturma
dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})



In [22]:
from transformers import AutoTokenizer

# Tokenizer yükleme
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    questions = examples["question"]
    contexts = examples["context"]
    answers = examples["answers"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    start_positions = []
    end_positions = []

    for i in range(len(answers)):
        if len(answers[i]) > 0:  # Eğer cevap varsa
            start_char = answers[i][0]["answer_start"]
            end_char = start_char + len(answers[i][0]["text"])

            # Karakter pozisyonlarını token pozisyonlarına çevir
            start_token = inputs.char_to_token(i, start_char)

            # Eğer start_token bulunamazsa, geçerli token pozisyonu eklemeyelim
            if start_token is None:
                start_positions.append(0)
                end_positions.append(0)
                continue

            # Cevap token uzunluğunu kullanarak end token'ı hesapla
            end_token = start_token + len(answers[i][0]["text"].split()) - 1

            # Eğer end_token bulunamazsa, geçerli token pozisyonu eklemeyelim
            if end_token is None:
                start_positions.append(0)
                end_positions.append(0)
                continue

            start_positions.append(start_token)
            end_positions.append(end_token)
        else:
            # Eğer cevap yoksa başlangıç ve bitiş pozisyonlarını 0 olarak ayarla
            start_positions.append(0)
            end_positions.append(0)

    # Tokenize edilen verilere başlangıç ve bitiş pozisyonlarını ekle
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

# Dataset'e uygulama
tokenized_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/6560 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

In [63]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Modeli yükleme
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Eğitimi başlat
trainer.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-63-3a73321121d2>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.522809
2,0.719500,0.482354
3,0.512100,0.460386
4,0.439300,0.513222
5,0.354600,0.528144
6,0.354600,0.602433
7,0.301700,0.636295


TrainOutput(global_step=2870, training_loss=0.4373123514527643, metrics={'train_runtime': 4557.7449, 'train_samples_per_second': 10.075, 'train_steps_per_second': 0.63, 'total_flos': 1.199874706931712e+16, 'train_loss': 0.4373123514527643, 'epoch': 7.0})

In [64]:
# Test veri setinde değerlendirme
results = trainer.evaluate(tokenized_dataset["test"])
print(results)


{'eval_loss': 0.5928071737289429, 'eval_runtime': 21.8065, 'eval_samples_per_second': 37.603, 'eval_steps_per_second': 2.385, 'epoch': 7.0}


In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

# Model ve tokenizer'ı yükle
model_path = "/content/results/checkpoint-2870"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

def ask_question(question, context):
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # En yüksek puanlı token pozisyonlarını bulun
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    # Cevabı token pozisyonlarına göre decode et
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    return answer

# Kullanıcıdan soru al
question = input("Soru sorunuz: ")
context = "Apse (apse; abscess; abscessus), irinli yangı bölgesinde doku erimesi vardır, oluşan boşluğu irin doldurur. Genellikle piyojen bakterilerin neden olduğu fokal bir yangı türüdür. Vücudun herhangi bir yerinde bir doku ya da organda oluşan apse, kendini ağrı, kızarıklık ve şişikler yaparak belli eder. Bazı apseler kendiliğinden dışarı açılır ve akar. Apse'nin açılmadığı durumlarda bu işi ameliyatla yapmak gerekir. Absenin mutlak tedavisi drenaj, yani boşaltılmasıdır. Cerrahi girişimden önce antibiyotik uygulanması sepsisi önler"

# Soruyu sor ve cevabı al
answer = ask_question(question, context)
print("Cevap:", answer)


OSError: Incorrect path_or_model_id: '/content/results/checkpoint-2870'. Please provide either the path to a local folder or the repo_id of a model on the Hub.